In [1]:
!pip install -U "git+https://github.com/ab7289-tandon-nyu/csgy6953_DeepLearning_Midterm.git"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/ab7289-tandon-nyu/csgy6953_DeepLearning_Midterm.git to /tmp/pip-req-build-48ev9m93
  Running command git clone -q https://github.com/ab7289-tandon-nyu/csgy6953_DeepLearning_Midterm.git /tmp/pip-req-build-48ev9m93
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done


In [2]:
import torch
import torch.nn as nn
import torchvision
import torchvision.datasets as datasets
import torch.utils.data as data
import random
import numpy as np
from torchsummary import summary

import copy
import time

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
ROOT = '.data'
train_data = datasets.CIFAR10(root = ROOT, 
                              train = True, 
                              download = True)

Files already downloaded and verified


In [6]:
# Compute means and standard deviations along the R,G,B channel

means = train_data.data.mean(axis = (0,1,2)) / 255
stds = train_data.data.std(axis = (0,1,2)) / 255

In [7]:
from src.transforms import make_transforms

train_transforms, test_transforms = make_transforms(means, stds)

In [8]:
train_data = datasets.CIFAR10(ROOT, 
                              train = True, 
                              download = True, 
                              transform = train_transforms)

test_data = datasets.CIFAR10(ROOT, 
                             train = False, 
                             download = True, 
                             transform = test_transforms)

Files already downloaded and verified
Files already downloaded and verified


In [9]:
VALID_RATIO = 0.9

n_train_examples = int(len(train_data) * VALID_RATIO)
n_valid_examples = len(train_data) - n_train_examples

train_data, valid_data = data.random_split(train_data, 
                                           [n_train_examples, n_valid_examples])

In [10]:
valid_data = copy.deepcopy(valid_data)
valid_data.dataset.transform = test_transforms

In [11]:
BATCH_SIZE = 512

train_iterator = data.DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)

valid_iterator = data.DataLoader(valid_data, batch_size=BATCH_SIZE, shuffle=True)

test_iterator = data.DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=True)

**Define our Model

In [12]:
from src.model import ResNet, StemConfig
from src.utils import initialize_parameters, epoch_time

model_architecture = (
    (1, 128),
    (2, 128),
    (2, 128),
    (2, 128),
    (2, 196),
    (2, 196),
)

stem_config = StemConfig(num_channels=128, kernel_size=5, stride=1, padding=2)
model = ResNet(model_architecture, stem_config=stem_config, output_size=10)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


Need to run a dummy set of data to initialize the lazy modules before we can use torchsummary

In [13]:
inputs = torch.empty((512, 3, 32, 32))
inputs.normal_()
model = model.to(device)
y = model(inputs.to(device))
y.size()

torch.Size([512, 10])

In [14]:
model.apply(initialize_parameters)

ResNet(
  (stem): Sequential(
    (0): Conv2d(3, 128, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (classifier): Sequential(
    (0): AdaptiveAvgPool2d(output_size=1)
    (1): Flatten(start_dim=1, end_dim=-1)
    (2): Linear(in_features=196, out_features=10, bias=True)
  )
  (body): Sequential(
    (block_2): Sequential(
      (0): ResidualBlock(
        (conv1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (relu): ReLU(inplace=True)
        (out): ReLU(inplace=True)
        (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (block_3): Sequential(
      (0): ResidualBlock(
        (conv1): Conv2d

In [15]:
print(f"num params: {sum([p.numel() for p in model.parameters() if p.requires_grad]):,}")

num params: 4,845,850


In [16]:
from src.engine import train_one_epoch, evaluate

best_loss = float('inf')
EPOCHS  = 100
learning_rate = 1e-3
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [17]:
for epoch in range(1, EPOCHS+1):
    start = time.time()

    print(f"Epoch {epoch}")
    train_loss, train_acc = train_one_epoch(model, train_iterator, criterion, optimizer, device)
    train_mins, train_secs = epoch_time(start, time.time())

    print(f"\tTrain elapsed: {train_mins}:{train_secs}, loss: {train_loss:.4f}, acc: {train_acc * 100:.2f}%")

    start = time.time()
    val_loss, val_acc = evaluate(model, valid_iterator, criterion, device)
    val_mins, val_secs = epoch_time(start, time.time())

    print(f"\tValidation elapsed: {val_mins}:{val_secs}, loss: {val_loss:.4f}, acc: {val_acc * 100:.2f}%")

    if val_loss < best_loss:
        best_loss = val_loss
        torch.save(model.state_dict(), "/content/drive/MyDrive/Colab Notebooks/midterm/resnet_alex_485.pt")

Epoch 1
	Train elapsed: 1:5, loss: 1.9083, acc: 36.01%
	Validation elapsed: 0:2, loss: 1.4952, acc: 45.69%
Epoch 2
	Train elapsed: 1:4, loss: 1.3850, acc: 49.59%
	Validation elapsed: 0:2, loss: 1.3388, acc: 52.10%
Epoch 3
	Train elapsed: 1:4, loss: 1.2076, acc: 56.35%
	Validation elapsed: 0:2, loss: 1.1293, acc: 59.44%
Epoch 4
	Train elapsed: 1:5, loss: 1.0547, acc: 62.14%
	Validation elapsed: 0:2, loss: 1.2483, acc: 57.47%
Epoch 5
	Train elapsed: 1:5, loss: 0.9348, acc: 66.54%
	Validation elapsed: 0:2, loss: 0.9695, acc: 66.99%
Epoch 6
	Train elapsed: 1:5, loss: 0.8246, acc: 70.75%
	Validation elapsed: 0:2, loss: 0.8770, acc: 69.40%
Epoch 7
	Train elapsed: 1:5, loss: 0.7517, acc: 73.30%
	Validation elapsed: 0:2, loss: 0.7725, acc: 73.57%
Epoch 8
	Train elapsed: 1:5, loss: 0.6831, acc: 75.92%
	Validation elapsed: 0:2, loss: 0.7394, acc: 74.50%
Epoch 9
	Train elapsed: 1:5, loss: 0.6337, acc: 77.52%
	Validation elapsed: 0:2, loss: 0.6721, acc: 77.05%
Epoch 10
	Train elapsed: 1:5, loss: 0

## Evaluate the Model  

In [18]:
model.load_state_dict(torch.load("/content/drive/MyDrive/Colab Notebooks/midterm/resnet_alex_485.pt"))
test_loss, test_acc = evaluate(model.to(device), test_iterator, criterion, device)
print(f"Test Loss: {test_loss:.4f}\nTest Accuracy: {test_acc * 100:.2f}%")

Test Loss: 0.4518
Test Accuracy: 86.91%
